# Artificial Neural Networks - Forward Propagation

In this particular assignment, I am trying to implement the following contents.


Contents
1. Initalize a Network
2. Compute Weighted Sum at Each Node
3. Compute Node Activation
4. Forward Propagation

# Initialize a network

Let's start by formally defining the structure of the network. Here, we provide 2 inputs and 2 hidden layers. Each of the hidden layer comprises of two nodes. Finally, we have an output layer.

In [33]:
n = 2 # number of inputs
num_hidden_layers = 2 # number of hidden layers
m = [2, 2] # number of nodes in each hidden layer
num_nodes_output = 1 # number of nodes in the output layer


Now that we defined the structure of the network, let's go ahead and inititailize the weights and the biases in the network to random numbers. In order to be able to initialize the weights and the biases to random numbers, import the Numpy library is a pre-requisite. Then, an empty dictionary is used to initialize the network.  

In [34]:
import numpy as np # import the Numpy library

num_nodes_previous = n # number of nodes in the previous layer

network = {} # initialize network an an empty dictionary

# loop through each layer and randomly initialize the weights and biases associated with each node
# notice how we are adding 1 to the number of hidden layers in order to include the output layer
for layer in range(num_hidden_layers + 1): 
    
    # determine name of layer
    if layer == num_hidden_layers:
        layer_name = 'output'
        num_nodes = num_nodes_output
    else:
        layer_name = 'layer_{}'.format(layer + 1)
        num_nodes = m[layer]
    
    # initialize weights and biases associated with each node in the current layer
    network[layer_name] = {}
    for node in range(num_nodes):
        node_name = 'node_{}'.format(node+1)
        network[layer_name][node_name] = {
            'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
            'bias': np.around(np.random.uniform(size=1), decimals=2),
        }
    
    num_nodes_previous = num_nodes
    
print(network) # print network

{'layer_1': {'node_1': {'weights': array([0.92, 0.9 ]), 'bias': array([0.03])}, 'node_2': {'weights': array([0.96, 0.14]), 'bias': array([0.28])}}, 'layer_2': {'node_1': {'weights': array([0.61, 0.94]), 'bias': array([0.85])}, 'node_2': {'weights': array([0.  , 0.52]), 'bias': array([0.55])}}, 'output': {'node_1': {'weights': array([0.49, 0.77]), 'bias': array([0.16])}}}


Awesome! So now with the above code, we are able to initialize the weights and the biases pertaining to any network of any number of hidden layers and number of nodes in each layer. But let's put this code in a function so that we are able to repetitively execute all this code whenever we want to construct a neural network. 

In [35]:
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):
    
    num_nodes_previous = num_inputs # number of nodes in the previous layer

    network = {}
    
    # loop through each layer and randomly initialize the weights and biases associated with each layer
    for layer in range(num_hidden_layers + 1):
        
        if layer == num_hidden_layers:
            layer_name = 'output' # name last layer in the network output
            num_nodes = num_nodes_output
        else:
            layer_name = 'layer_{}'.format(layer + 1) # otherwise give the layer a number
            num_nodes = num_nodes_hidden[layer] 
        
        # initialize weights and bias for each node
        network[layer_name] = {}
        for node in range(num_nodes):
            node_name = 'node_{}'.format(node+1)
            network[layer_name][node_name] = {
                'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
                'bias': np.around(np.random.uniform(size=1), decimals=2),
            }
    
        num_nodes_previous = num_nodes

    return network # return the network

Use the initialize_network function to create a network that:

1. takes 5 inputs
2. has three hidden layers
3. has 3 nodes in the first layer, 2 nodes in the second layer, and 3 nodes in the third layer
4. has 1 node in the output layer

Lets call the network 'small_network'. The network will look like this:



In [36]:
### a network called small_network is created which contains 5 inputs, three hidden layers(3 nodes in the first layer, 2 nodes in the second layer, 
### and 3 nodes in the third layer), 1 node in the output layer. 
small_network = initialize_network(5, 3, [3, 2, 3], 1)
print(small_network)

{'layer_1': {'node_1': {'weights': array([0.76, 0.02, 0.14, 0.12, 0.31]), 'bias': array([0.67])}, 'node_2': {'weights': array([0.47, 0.82, 0.29, 0.73, 0.7 ]), 'bias': array([0.33])}, 'node_3': {'weights': array([0.33, 0.98, 0.62, 0.95, 0.77]), 'bias': array([0.83])}}, 'layer_2': {'node_1': {'weights': array([0.41, 0.45, 0.4 ]), 'bias': array([1.])}, 'node_2': {'weights': array([0.18, 0.96, 0.42]), 'bias': array([0.42])}}, 'layer_3': {'node_1': {'weights': array([0.46, 0.37]), 'bias': array([0.47])}, 'node_2': {'weights': array([0.04, 0.08]), 'bias': array([0.73])}, 'node_3': {'weights': array([0.64, 0.03]), 'bias': array([0.3])}}, 'output': {'node_1': {'weights': array([0.22, 0.06, 0.52]), 'bias': array([0.42])}}}


# Compute Weighted Sum at Each Node

The weighted sum at each node is computed as the dot product of the inputs and the weights plus the bias. So let's create a function called compute_weighted_sum that does just that.


In [37]:
def compute_weighted_sum(inputs, weights, bias):
    return np.sum(inputs * weights) + bias

Let's generate 5 inputs that we can feed to **small_network**.

In [38]:
from random import seed
import numpy as np

np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('The inputs to the network are {}'.format(inputs))

The inputs to the network are [0.15 0.74 0.26 0.53 0.01]


Use the compute_weighted_sum function to compute the weighted sum at the first node in the first hidden layer. I have also calculate it by hands which is shown in below figure to verify the result.

In [39]:
### type your answer here
weighted_sum = compute_weighted_sum(inputs, small_network['layer_1']['node_1']['weights'], small_network['layer_1']['node_1']['bias'])
print(weighted_sum)

[0.9019]


# Compute Node Activation

Recall that the output of each node is simply a non-linear tranformation of the weighted sum. We use activation functions for this mapping. Let's use the sigmoid function as the activation function here. So let's define a function that takes a weighted sum as input and returns the non-linear transformation of the input using the sigmoid function.


In [40]:
def node_activation(weighted_sum):
    return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

Use the node_activation function to compute the output of the first node in the first hidden layer.

In [41]:
### type your answer here
print(node_activation(weighted_sum))

[0.7113398]


# Forward Propagation

The final piece of building a neural network that can perform predictions is to put everything together. So let's create a function that applies the compute_weighted_sum and node_activation functions to each node in the network and propagates the data all the way to the output layer and outputs a prediction for each node in the output layer.

The way we are going to accomplish this is through the following procedure:

1. Start with the input layer as the input to the first hidden layer.
2. Compute the weighted sum at the nodes of the current layer.
3. Compute the output of the nodes of the current layer.
4. Set the output of the current layer to be the input to the next layer.
5. Move to the next layer in the network.
6. Repeat steps 2 - 4 until we compute the output of the output layer.



In [42]:
def forward_propagate(network, inputs):
    
    layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer
    
    for layer in network:
        
        layer_data = network[layer]
        
        layer_outputs = [] 
        for layer_node in layer_data:
        
            node_data = layer_data[layer_node]
        
            # compute the weighted sum and the output of each node at the same time 
            node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
            layer_outputs.append(np.around(node_output[0], decimals=4))
            
        if layer != 'output':
            print('The outputs of the nodes in hidden layer number {} is {}'.format(layer.split('_')[1], layer_outputs))
    
        layer_inputs = layer_outputs # set the output of this layer to be the input to next layer

    network_predictions = layer_outputs
    return network_predictions

Use the forward_propagate function to compute the prediction of our small network

In [43]:
### type your answser here
prediction = forward_propagate(small_network, inputs)
print(prediction)


The outputs of the nodes in hidden layer number 1 is [0.7113, 0.8141, 0.907]
The outputs of the nodes in hidden layer number 2 is [0.883, 0.8469]
The outputs of the nodes in hidden layer number 3 is [0.7667, 0.697, 0.709]
[0.7309]


Now some experimentation have been done to create new feedforward networks. 

At first A network is implemented which contains 10 inputs, three hidden layers(4 nodes in the first layer, 5 nodes in the second layer, 
and 2 nodes in the third layer), 1 node in the output layer. The network is call 'New_network_1'. The network is initialized by using the function 'initialize_network'. Then inputs are created and then use the function 'forward_propagate'. 

In [44]:
### create a network


from random import seed
import numpy as np

np.random.seed(12)
inputs = np.around(np.random.uniform(size=10), decimals=2)

print('The inputs to the network are {}'.format(inputs))
New_network_1 = initialize_network(10, 3, [4, 5, 2], 1)

prediction = forward_propagate(New_network_1, inputs)
print(prediction)

The inputs to the network are [0.15 0.74 0.26 0.53 0.01 0.92 0.9  0.03 0.96 0.14]
The outputs of the nodes in hidden layer number 1 is [0.9758, 0.9319, 0.9643, 0.8912]
The outputs of the nodes in hidden layer number 2 is [0.8083, 0.8493, 0.9465, 0.9477, 0.8472]
The outputs of the nodes in hidden layer number 3 is [0.979, 0.9561]
[0.8975]


Another network is implemented which contains 10 inputs, five hidden layers(4 nodes in the first layer, 5 nodes in the second layer, 5 nodes in the third layer, 3 nodes in the forth layer, and 2 nodes in the fifth layer), 1 node in the output layer. The network is call 'New_network_2'. The network is initialized by using the function 'initialize_network'. Then inputs are created and then use the function 'forward_propagate'. 

In [45]:
### create a network



from random import seed
import numpy as np

np.random.seed(12)
inputs = np.around(np.random.uniform(size=10), decimals=2)

print('The inputs to the network are {}'.format(inputs))
New_network_2 = initialize_network(10, 5, [4, 5, 5, 3, 2], 1)

prediction = forward_propagate(New_network_2, inputs)
print(prediction)

The inputs to the network are [0.15 0.74 0.26 0.53 0.01 0.92 0.9  0.03 0.96 0.14]
The outputs of the nodes in hidden layer number 1 is [0.9758, 0.9319, 0.9643, 0.8912]
The outputs of the nodes in hidden layer number 2 is [0.8083, 0.8493, 0.9465, 0.9477, 0.8472]
The outputs of the nodes in hidden layer number 3 is [0.979, 0.9561, 0.9709, 0.9451, 0.9265]
The outputs of the nodes in hidden layer number 4 is [0.9171, 0.9242, 0.9231]
The outputs of the nodes in hidden layer number 5 is [0.8879, 0.7071]
[0.6493]


Thank you for completing this assignment!